In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from requests_html import HTML
import glob
import warnings

In [2]:
df = pd.read_csv("Data/omdb_movies_data.csv", low_memory=False)
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Birmingham,1896.0,NaN,NaN,61 min,"Documentary, Short",NaN,NaN,NaN,The most successful girl group of the Nineties...,...,NaN,NaN,NaN,tt2210499,movie,NaN,NaN,NaN,NaN,True
1,Reproduction of the Corbett and Fitzsimmons Fight,1897.0,NaN,NaN,60 min,"News, Sport",NaN,NaN,NaN,NaN,...,NaN,3.7,27,tt0229676,movie,NaN,NaN,NaN,NaN,True
2,Dressing Paper Dolls,1898.0,NaN,01 Jul 1898,61 min,Documentary,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0235357,movie,NaN,NaN,NaN,NaN,True
3,"Buck Dance, Ute Indians",1898.0,NaN,01 Feb 1898,60 min,Documentary,James H. White,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0221040,movie,NaN,NaN,NaN,NaN,True
4,69th Regiment Passing in Review,1898.0,NaN,01 Jul 1898,61 min,Documentary,NaN,NaN,NaN,"""This well-known regiment of New York City mil...",...,NaN,NaN,NaN,tt0236940,movie,NaN,NaN,NaN,NaN,True


In [3]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(458459, 25)
(438120, 25)


In [4]:
df = df.rename(columns={'imdbID':'IMDB ID','imdbRating':'Rating','imdbVotes':'Votes'})
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,Rating,Votes,IMDB ID,Type,DVD,BoxOffice,Production,Website,Response
0,Birmingham,1896.0,NaN,NaN,61 min,"Documentary, Short",NaN,NaN,NaN,The most successful girl group of the Nineties...,...,NaN,NaN,NaN,tt2210499,movie,NaN,NaN,NaN,NaN,True
1,Reproduction of the Corbett and Fitzsimmons Fight,1897.0,NaN,NaN,60 min,"News, Sport",NaN,NaN,NaN,NaN,...,NaN,3.7,27,tt0229676,movie,NaN,NaN,NaN,NaN,True
2,Dressing Paper Dolls,1898.0,NaN,01 Jul 1898,61 min,Documentary,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0235357,movie,NaN,NaN,NaN,NaN,True
3,"Buck Dance, Ute Indians",1898.0,NaN,01 Feb 1898,60 min,Documentary,James H. White,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0221040,movie,NaN,NaN,NaN,NaN,True
4,69th Regiment Passing in Review,1898.0,NaN,01 Jul 1898,61 min,Documentary,NaN,NaN,NaN,"""This well-known regiment of New York City mil...",...,NaN,NaN,NaN,tt0236940,movie,NaN,NaN,NaN,NaN,True


In [5]:
df.dtypes

Title          object
Year          float64
Rated          object
Released       object
Runtime        object
Genre          object
Director       object
Writer         object
Actors         object
Plot           object
Language       object
Country        object
Awards         object
Poster         object
Ratings        object
Metascore     float64
Rating        float64
Votes          object
IMDB ID        object
Type           object
DVD            object
BoxOffice      object
Production     object
Website        object
Response       object
dtype: object

Release date

In [6]:
df['Released']=pd.to_datetime(df['Released'])
df['Released'].head()

0          NaT
1          NaT
2   1898-07-01
3   1898-02-01
4   1898-07-01
Name: Released, dtype: datetime64[ns]

In [7]:
rdate = pd.DataFrame()

In [8]:
rdate['Month'] = df['Released'].dt.month 
rdate['Monthname']= df['Released'].dt.strftime("%B")
rdate['Day'] = df['Released'].dt.day
rdate['Weekday']= df['Released'].dt.day_name()
print(rdate.head())
df = df.join(rdate)
print("-------------")
print(df.head())

   Month Monthname  Day  Weekday
0    NaN       NaN  NaN      NaN
1    NaN       NaN  NaN      NaN
2    7.0      July  1.0   Friday
3    2.0  February  1.0  Tuesday
4    7.0      July  1.0   Friday
-------------
                                               Title    Year Rated   Released  \
0                                         Birmingham  1896.0   NaN        NaT   
1  Reproduction of the Corbett and Fitzsimmons Fight  1897.0   NaN        NaT   
2                               Dressing Paper Dolls  1898.0   NaN 1898-07-01   
3                            Buck Dance, Ute Indians  1898.0   NaN 1898-02-01   
4                    69th Regiment Passing in Review  1898.0   NaN 1898-07-01   

  Runtime               Genre        Director Writer Actors  \
0  61 min  Documentary, Short             NaN    NaN    NaN   
1  60 min         News, Sport             NaN    NaN    NaN   
2  61 min         Documentary             NaN    NaN    NaN   
3  60 min         Documentary  James H. White    

In [9]:
df.dtypes

Title                 object
Year                 float64
Rated                 object
Released      datetime64[ns]
Runtime               object
Genre                 object
Director              object
Writer                object
Actors                object
Plot                  object
Language              object
Country               object
Awards                object
Poster                object
Ratings               object
Metascore            float64
Rating               float64
Votes                 object
IMDB ID               object
Type                  object
DVD                   object
BoxOffice             object
Production            object
Website               object
Response              object
Month                float64
Monthname             object
Day                  float64
Weekday               object
dtype: object

In [10]:
df2 = df['Runtime'].str.split(n=1, expand=True)
df2.columns = ['Runtime{}'.format(x+1) for x in df2.columns]
df2 = df2.rename(columns={'Runtime1':"Duration"})
df2 = df2.drop(columns = ['Runtime2'])
df = df.join(df2)
df.head(5)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,DVD,BoxOffice,Production,Website,Response,Month,Monthname,Day,Weekday,Duration
0,Birmingham,1896.0,NaN,NaT,61 min,"Documentary, Short",NaN,NaN,NaN,The most successful girl group of the Nineties...,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,61
1,Reproduction of the Corbett and Fitzsimmons Fight,1897.0,NaN,NaT,60 min,"News, Sport",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,60
2,Dressing Paper Dolls,1898.0,NaN,1898-07-01,61 min,Documentary,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,7.0,July,1.0,Friday,61
3,"Buck Dance, Ute Indians",1898.0,NaN,1898-02-01,60 min,Documentary,James H. White,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,2.0,February,1.0,Tuesday,60
4,69th Regiment Passing in Review,1898.0,NaN,1898-07-01,61 min,Documentary,NaN,NaN,NaN,"""This well-known regiment of New York City mil...",...,NaN,NaN,NaN,NaN,True,7.0,July,1.0,Friday,61


In [11]:
df = df.drop(columns=['Rated','Writer','DVD','Plot','Website','Runtime','Response', 'Metascore', 'Ratings', 'Type', 'Awards'])

Removing null values

In [12]:
df.shape

(438120, 19)

In [13]:
df.dtypes

Title                 object
Year                 float64
Released      datetime64[ns]
Genre                 object
Director              object
Actors                object
Language              object
Country               object
Poster                object
Rating               float64
Votes                 object
IMDB ID               object
BoxOffice             object
Production            object
Month                float64
Monthname             object
Day                  float64
Weekday               object
Duration              object
dtype: object

In [14]:
print(df['Title'].isnull().sum())
df['Title'] = df['Title'].fillna('No Title')
print(df['Title'].isnull().sum())

2
0


In [15]:
convert_dict = {'Year':int}
df['Year'] = df['Year'].fillna(0)
df = df.astype(convert_dict)
print(df['Year'].isnull().sum())


0


In [16]:
print(df['Released'].isnull().sum())
df['Released'] = df['Released'].fillna('01-01-1800')
print(df['Released'].isnull().sum())

141594
0


In [17]:
print(df['Genre'].isnull().sum())
df['Genre'] = df['Genre'].fillna('No Genre')
print(df['Genre'].isnull().sum())

70961
0


In [18]:
print(df['Director'].isnull().sum())
df['Director'] = df['Director'].fillna('Director Not Mentioned')
print(df['Director'].isnull().sum())

28001
0


In [19]:
print(df['Actors'].isnull().sum())
df['Actors'] = df['Actors'].fillna('Actors Not Mentioned')
print(df['Actors'].isnull().sum())

72903
0


In [20]:
print(df['Language'].isnull().sum())
df['Language'] = df['Language'].fillna('No Language')
print(df['Language'].isnull().sum())

40601
0


In [21]:
print(df['Country'].isnull().sum())
df['Country'] = df['Country'].fillna('No Country')
print(df['Country'].isnull().sum())

16680
0


In [22]:
print(df['Poster'].isnull().sum())
df['Poster'] = df['Poster'].fillna('No Poster')
print(df['Poster'].isnull().sum())

209428
0


In [23]:
df['Rating'] = pd.to_numeric(df['Rating'],errors='coerce')
print(df['Rating'].isnull().sum())
df['Rating'] = df['Rating'].fillna(0.0)
print(df['Rating'].isnull().sum())

232323
0


In [24]:
print(df['Votes'].isnull().sum())
df['Votes'] = pd.to_numeric(df['Votes'],errors='coerce')
print(df['Votes'].isnull().sum())
df['Votes'] = df['Votes'].fillna(0.0)
print(df['Votes'].isnull().sum())

232375
260521
0


In [25]:
print(df['IMDB ID'].isnull().sum())
df['IMDB ID'] = df['IMDB ID'].fillna('No imdbID')
print(df['IMDB ID'].isnull().sum())

2
0


In [26]:
print(df['BoxOffice'].isnull().sum())
df['BoxOffice'] = df['BoxOffice'].fillna('No BoxOffice')
print(df['BoxOffice'].isnull().sum())

423954
0


In [27]:
print(df['Production'].isnull().sum())
df['Production'] = df['Production'].fillna('No Production')
print(df['Production'].isnull().sum())

401071
0


In [28]:
# Month
df['Month'] = pd.to_numeric(df['Month'],errors='coerce')
print(df['Month'].isnull().sum())
df['Month'] = df['Month'].fillna(0.0)
print(df['Month'].isnull().sum())

141594
0


In [29]:
# MonthName
print(df['Monthname'].isnull().sum())
df['MonthName'] = df['Monthname'].fillna('No Month')
print(df['Monthname'].isnull().sum())

141594
141594


In [30]:
# Day
df['Day'] = pd.to_numeric(df['Day'],errors='coerce')
print(df['Day'].isnull().sum())
df['Day'] = df['Day'].fillna(0.0)
print(df['Day'].isnull().sum())

141594
0


In [31]:
# Weekday
print(df['Weekday'].isnull().sum())
df['Weekday'] = df['Weekday'].fillna('No Weekday')
print(df['Weekday'].isnull().sum())

141594
0


In [32]:
# df['Duration'] = df['Duration'].astype(int)
df['Duration'] = pd.to_numeric(df['Duration'],errors='coerce')
print(df['Duration'].isnull().sum())
df['Duration'] = df['Duration'].fillna(0)
print(df['Duration'].isnull().sum())
df['Duration'] = df['Duration'].astype(int)

157235
0


In [33]:
df = df.drop(columns=['Monthname', 'Month'])

In [34]:
df.isnull().sum()

Title         0
Year          0
Released      0
Genre         0
Director      0
Actors        0
Language      0
Country       0
Poster        0
Rating        0
Votes         0
IMDB ID       0
BoxOffice     0
Production    0
Day           0
Weekday       0
Duration      0
MonthName     0
dtype: int64

In [35]:
df.to_csv("Data/analysed_omdb_movies_data.csv",index=None)

Analysis after getting tmdb and boxofficemojo data

In [36]:
# Appending all the years files into single file
path = r'C:/Users/madhu/Desktop/Final/Data/Years'
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    li.append(df1)

df = pd.concat(li, axis=0, ignore_index=True)
print(df.shape)
df = df.drop_duplicates()
df = df.rename(columns={'Release Group':"Title"})
df = df.drop(columns = ['%','Foreign','%.1','Rank','Domestic'])
print(df.shape)
df.to_csv("Data/boxofficemojo_data.csv",index=False)

(8240, 7)
(8240, 2)


In [37]:
# Appending the omdb analysed data, tmdb data and boxoffice mojo data into a single file
df1 = pd.read_csv("Data/analysed_omdb_movies_data.csv")
print(df1.shape)
df2 = df1.drop_duplicates()
print(df2.shape)
print(df2["IMDB ID"].nunique())
df3 = df2.drop_duplicates(subset=["IMDB ID"])
print(df3.shape)
df4 = pd.read_csv("Data/boxofficemojo_data.csv")
print(df4.shape)
df5 = df4.drop_duplicates()
print(df5.shape)
df6 = pd.read_csv("Data/tmdb_movies_data.csv")
print(df6.shape)
df7 = df6.drop_duplicates()
print(df7.shape)
df = pd.DataFrame()
df = df3.merge(df5, on='Title', how="left")
print(df.shape)
df = df.merge(df7, on='IMDB ID', how="left")

(438120, 18)
(435993, 18)
414305
(414305, 18)
(8240, 2)
(8240, 2)
(414495, 5)
(414495, 5)
(415276, 19)


In [38]:
warnings.filterwarnings('ignore')
# getting the conversion value of usd to inr
url = 'https://api.exchangerate-api.com/v4/latest/USD'
data = requests.get(url).json()
currencies = data['rates']
indiancurrency = currencies['INR']

df["BoxOffice"] = df["BoxOffice"].str.replace(r'\D', '')
df['BoxOffice'] = df['BoxOffice'].replace(np.NaN, 0)
df["Worldwide"] = df["Worldwide"].str.replace(r'\D', '')
df['Worldwide'] = df['Worldwide'].replace(np.NaN, 0)
df["BoxOffice"] = pd.to_numeric(df['BoxOffice'],errors='coerce')
df["Worldwide"] = pd.to_numeric(df['Worldwide'],errors='coerce')

#Inflation is done here
df["Budget"] = df["Budget"] *indiancurrency
df["Collections"] = (df['BoxOffice']*indiancurrency)+(df['Revenue']*indiancurrency)+(df['Worldwide']*indiancurrency)
df = df.drop(columns=['BoxOffice','Revenue','Worldwide'])
df = df.drop_duplicates(subset=['IMDB ID'])


In [ ]:
df = df.drop(columns=['Certificate'])

In [39]:
df['Profit'] = df['Collections'] - df['Budget']

In [40]:
 # Budget
df['Budget'] = df['Budget'].fillna(0.0)

# Popularity
df['Popularity'] = df['Popularity'].fillna(0.0)

# Collections
df['Collections'] = df['Collections'].fillna(0.0)

# Profit
df['Profit'] = df['Profit'].fillna(0.0)

In [41]:
df.isnull().sum()

Title          0
Year           0
Released       0
Genre          0
Director       0
Actors         0
Language       0
Country        0
Poster         0
Rating         0
Votes          0
IMDB ID        0
Production     0
Day            0
Weekday        0
Duration       0
MonthName      0
Budget         0
Popularity     0
Certificate    0
Collections    0
Profit         0
dtype: int64

In [42]:
# the final data is here
df.to_csv("Data/movies_data.csv",index=False) 

Data Wrangling

In [46]:
movies_data = pd.read_csv("Data/movies_data.csv", low_memory=False)
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414305 entries, 0 to 414304
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Title        414305 non-null  object 
 1   Year         414305 non-null  int64  
 2   Released     414305 non-null  object 
 3   Genre        414305 non-null  object 
 4   Director     414305 non-null  object 
 5   Actors       414305 non-null  object 
 6   Language     414305 non-null  object 
 7   Country      414305 non-null  object 
 8   Poster       414305 non-null  object 
 9   Rating       414305 non-null  float64
 10  Votes        414305 non-null  float64
 11  IMDB ID      414305 non-null  object 
 12  Production   414305 non-null  object 
 13  Day          414305 non-null  float64
 14  Weekday      414305 non-null  object 
 15  Duration     414305 non-null  int64  
 16  MonthName    414305 non-null  object 
 17  Budget       414305 non-null  float64
 18  Popularity   414305 non-

In [47]:
movies_data.describe()

,Year,Rating,Votes,Day,Duration,Budget,Popularity,Collections,Profit
count,414305.000000,414305.000000,414305.000000,414305.000000,414305.000000,4.143050e+05,414305.000000,4.143050e+05,4.143050e+05
mean,1989.181726,2.855588,50.670762,9.316624,56.615945,2.114081e+09,0.909998,5.300996e+09,3.569416e+09
std,37.244314,3.185304,138.233716,10.159833,47.763874,3.060295e+10,11.419470,1.095551e+11,8.891024e+10
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,-8.615340e+11
25%,1970.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
50%,2004.000000,0.000000,0.000000,5.000000,70.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
75%,2014.000000,6.200000,22.000000,18.000000,92.000000,0.000000e+00,0.600000,0.000000e+00,0.000000e+00
max,2028.000000,10.000000,999.000000,31.000000,950.000000,5.351620e+12,4006.705000,1.524362e+13,1.391121e+13
